In [ ]:
%matplotlib notebook

import sys, os 
print(os.path.abspath('.'))
sys.path.append(os.path.join('..','..')) 

import pylab
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy
import scipy
import scipy.special

from matplotlib import pylab
from matplotlib import pyplot as plt

from crocodile.synthesis import *
from crocodile.simulate import *
from msclean import *

def stackshow(stack):
    plt.clf()
    nscales=stack.shape[2]
    for iscale in range(nscales):
        ny=int(numpy.sqrt(nscales))
        nx=ny
        plt.subplot(nx,ny,iscale+1)
        plt.imshow(stack[:,:,iscale],origin='lower',cmap='Greys')

In [ ]:
vlas=numpy.genfromtxt("../../data/models/VLA_A_hor_xyz.txt", delimiter=",")
vobs=xyz_to_baselines(vlas, numpy.arange(0,numpy.pi,0.1) ,  numpy.pi/4)
yy=simulate_point(vobs/5, 0.01, 0.01)

In [ ]:
ps, vs = sortw(vobs/5, yy)
wstep=2500
dirty,psf=doimg(2*0.025, 2*15000, ps, vs, lambda *x: wslicimg(*x, wstep=wstep, Qpx=1))

In [ ]:
scales=[1,3,10]
ps, vs, cc, rres=majorcycle(2*0.025, 2*15000, vobs/5, yy, 0.1, 2, 100, wstep, scales)
    

In [ ]:
plt.clf()
plt.subplot(1, 2, 1)
plt.imshow(numpy.sqrt(numpy.fabs(cc)), origin='lower')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(rres, origin='lower')
plt.colorbar()

plt.show()